### Imports and images

In [1]:
import os
import cv2
from jita.tools.ollama_ocr import OllamaOCR
from jita.tools.zone_detector import SmartFlyerDetector
from jita.tools.zone_detector_v2 import UltimateFlyerDetector
from jita.settings.config import CASA_LEY_DATA, WINDOWED_CASA_LEY

folleto = list(CASA_LEY_DATA.glob("*.jpg"))[0]

In [36]:
print("=" * 70)
print("DETECTOR INTELIGENTE DE ZONAS EN FOLLETOS")
print("=" * 70)

# MÉTODO 1: Detección por espacios blancos (RECOMENDADO para tu folleto)
print("\n🎯 MÉTODO 1: Detección por Espacios Blancos")
print("-" * 70)

detector1 = SmartFlyerDetector(
    method='whitespace',           # Detecta espacios blancos naturales
    min_zone_height_percent=5,     # Zona mínima: 5% de altura
    min_zone_width_percent=25,     # Ancho mínimo: 15%
    padding_percent=5,             # 3% de padding
    whitespace_threshold=250       # Umbral de "blanco" (ajusta si necesario)
)

zones1 = detector1.detect_zones(
    folleto,
    f"{WINDOWED_CASA_LEY}/whitespace",
    debug=True
)

print("\n📋 Zonas detectadas:")
for zone in zones1:
    print(f"  Z{zone['id']:2d} | {zone['width']:4d}x{zone['height']:4d}px | {os.path.basename(zone['path'])}")


print("\n\n💡 TIPS DE AJUSTE:")
print("-" * 70)
print("Si las zonas son muy grandes:")
print("  → Disminuye 'min_zone_height_percent' a 3-4")
print("\nSi se pierden zonas pequeñas:")
print("  → Aumenta 'min_zone_height_percent' a 7-10")
print("\nSi se corta texto:")
print("  → Aumenta 'padding_percent' a 5-8")
print("\nSi detecta demasiados espacios blancos:")
print("  → Disminuye 'whitespace_threshold' a 230")
print("\nSi no detecta suficientes espacios:")
print("  → Aumenta 'whitespace_threshold' a 250")

DETECTOR INTELIGENTE DE ZONAS EN FOLLETOS

🎯 MÉTODO 1: Detección por Espacios Blancos
----------------------------------------------------------------------
📸 Imagen cargada: 3247x1774px

🔍 Método: Detección por espacios blancos...
   ✓ 1 bloques de contenido detectados
   ⚠️ Pocos bloques horizontales, aplicando subdivisión vertical...
📊 Visualización guardada: 3_final_zones.jpg

✅ Total: 1 zonas detectadas y guardadas

📋 Zonas detectadas:
  Z 1 | 3247x1774px | zone_01.jpg


💡 TIPS DE AJUSTE:
----------------------------------------------------------------------
Si las zonas son muy grandes:
  → Disminuye 'min_zone_height_percent' a 3-4

Si se pierden zonas pequeñas:
  → Aumenta 'min_zone_height_percent' a 7-10

Si se corta texto:
  → Aumenta 'padding_percent' a 5-8

Si detecta demasiados espacios blancos:
  → Disminuye 'whitespace_threshold' a 230

Si no detecta suficientes espacios:
  → Aumenta 'whitespace_threshold' a 250


https://medium.com/@amit25173/clustering-in-image-processing-explained-2cfc55ccae15

In [2]:

print("=" * 70)

# CONFIGURACIÓN RECOMENDADA
detector = UltimateFlyerDetector(
    use_clustering=True,           # Activar clustering
    clustering_method='kmeans',    # 'dbscan' o 'kmeans'
    target_zones='auto',           # Detección automática
    padding_percent=1.5,             # 4% padding
    min_zone_area_percent=1       # Zona mínima 2%
)

zones = detector.detect_zones(
    folleto,
    "output_ultimate",
    debug=True
)

print("\n" + "=" * 70)
print("📋 ZONAS DETECTADAS:")
print("=" * 70)
for zone in zones:
    print(f"  Z{zone['id']:2d} | {zone['width']:4d}x{zone['height']:4d}px | {os.path.basename(zone['path'])}")

print("\n💡 Archivos generados:")
print("  1_preprocessed.jpg - Preprocesamiento")
print("  2_projection_*.png - Gráficas de proyección")
print("  3_clusters.jpg - Visualización de clusters")
print("  4_final_zones.jpg - Zonas finales marcadas")
print("  zone_*.jpg - Recortes finales")

📸 Imagen: 3247x1774px (ratio: 1.83)

🧠 Estrategia: HORIZONTAL_FIRST

🔧 Preprocesamiento...

🔍 Fase 1: Detección primaria (HORIZONTAL_FIRST)...
   → Detectando columnas verticales...
   ✓ 2 zonas primarias detectadas

🔍 Fase 2: Subdivisión inteligente...
   ✓ 2 zonas tras subdivisión

🎯 Fase 3: Clustering (kmeans)...
   ✓ 2 clusters detectados
   ✓ 4 zonas tras clustering

🧹 Fase 4: Limpieza de zonas...
   ✓ 2 zonas finales tras limpieza

💾 Fase 5: Guardando zonas...

✅ COMPLETADO: 2 zonas listas

📋 ZONAS DETECTADAS:
  Z 1 | 1668x1774px | zone_01.jpg
  Z 2 | 1648x1774px | zone_02.jpg

💡 Archivos generados:
  1_preprocessed.jpg - Preprocesamiento
  2_projection_*.png - Gráficas de proyección
  3_clusters.jpg - Visualización de clusters
  4_final_zones.jpg - Zonas finales marcadas
  zone_*.jpg - Recortes finales


In [42]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans

class AdvancedKMeansCutter:
    """
    Utiliza K-Means clustering para segmentar folletos de forma inteligente.
    1. Determina automáticamente el número óptimo de zonas (k).
    2. Aplica un padding inteligente que evita el solapamiento entre recortes.
    """

    def _find_optimal_k_elbow(self, points, max_k=8):
        """
        Encuentra el número óptimo de clusters (k) usando el método del codo.
        """
        if len(points) < max_k:
            max_k = len(points)
            
        inertias = []
        k_range = range(2, max_k + 1)
        
        print(f"   -> Buscando k óptima entre 2 y {max_k}...")
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
            kmeans.fit(points)
            inertias.append(kmeans.inertia_)
        
        # Calcular la segunda derivada para encontrar el "codo"
        # El codo es el punto de máxima curvatura.
        if len(inertias) < 2: return 2 # Default
        
        diff1 = np.diff(inertias, n=1)
        diff2 = np.diff(inertias, n=2)

        # El índice del k óptimo es donde la segunda derivada (aceleración) es máxima.
        # Sumamos 2 porque el rango de k empieza en 2.
        optimal_k_index = np.argmax(diff2) if len(diff2) > 0 else 0
        optimal_k = k_range[optimal_k_index + 1]

        print(f"   -> 'Codo' detectado. K óptima = {optimal_k}")
        return optimal_k

    def _adjust_padding_to_prevent_overlap(self, zones, padding):
        """
        Ajusta el padding de cada zona para que no se solape con sus vecinas.
        """
        adjusted_zones = []
        for i, zone1 in enumerate(zones):
            x1, y1, w1, h1 = zone1
            max_pad_x = padding
            max_pad_y = padding

            for j, zone2 in enumerate(zones):
                if i == j: continue
                x2, y2, w2, h2 = zone2
                
                # Calcular distancia horizontal
                if x1 + w1 <= x2: # Z2 está a la derecha de Z1
                    dist_x = x2 - (x1 + w1)
                    max_pad_x = min(max_pad_x, dist_x // 2)
                elif x2 + w2 <= x1: # Z2 está a la izquierda de Z1
                    dist_x = x1 - (x2 + w2)
                    max_pad_x = min(max_pad_x, dist_x // 2)
                
                # Calcular distancia vertical
                if y1 + h1 <= y2: # Z2 está debajo de Z1
                    dist_y = y2 - (y1 + h1)
                    max_pad_y = min(max_pad_y, dist_y // 2)
                elif y2 + h2 <= y1: # Z2 está encima de Z1
                    dist_y = y1 - (y2 + h2)
                    max_pad_y = min(max_pad_y, dist_y // 2)

            adjusted_zones.append((x1, y1, w1, h1, max_pad_x, max_pad_y))
            
        return adjusted_zones

    def cut(self, image_path, output_folder, min_contour_area=100, default_padding=20, debug=False):
        os.makedirs(output_folder, exist_ok=True)
        img_color = cv2.imread(image_path)
        if img_color is None:
            raise FileNotFoundError(f"No se pudo cargar la imagen: {image_path}")
        h_img, w_img, _ = img_color.shape
        
        print("📸 Imagen cargada. Iniciando detección de zonas...")

        # --- 1. Encontrar puntos de contenido ---
        gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(cv2.GaussianBlur(gray, (5, 5), 0), 255, 
                                       cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                       cv2.THRESH_BINARY_INV, 11, 4)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        content_points, content_boxes = [], []
        for cnt in contours:
            if cv2.contourArea(cnt) > min_contour_area:
                x, y, w, h = cv2.boundingRect(cnt)
                content_points.append([x + w//2, y + h//2])
                content_boxes.append([x, y, x+w, y+h])

        if not content_points:
            print("⚠️ No se encontraron puntos de contenido.")
            return []

        points_np = np.array(content_points)

        # --- 2. Determinar k y aplicar K-Means ---
        optimal_k = self._find_optimal_k_elbow(points_np)
        kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init='auto').fit(points_np)
        labels = kmeans.labels_

        # --- 3. Generar zonas iniciales y ajustar padding ---
        initial_zones = []
        for i in range(optimal_k):
            cluster_indices = np.where(labels == i)[0]
            if len(cluster_indices) == 0: continue
            cluster_boxes = [content_boxes[j] for j in cluster_indices]
            min_x = min(b[0] for b in cluster_boxes)
            min_y = min(b[1] for b in cluster_boxes)
            max_x = max(b[2] for b in cluster_boxes)
            max_y = max(b[3] for b in cluster_boxes)
            initial_zones.append((min_x, min_y, max_x - min_x, max_y - min_y))

        adjusted_padded_zones = self._adjust_padding_to_prevent_overlap(initial_zones, default_padding)

        # --- 4. Recortar, guardar y visualizar ---
        cropped_paths = []
        img_debug = img_color.copy()
        adjusted_padded_zones.sort(key=lambda z: (z[1], z[0])) # Ordenar para nombrar consistentemente

        for i, (x, y, w, h, pad_x, pad_y) in enumerate(adjusted_padded_zones):
            x_pad = max(0, x - pad_x)
            y_pad = max(0, y - pad_y)
            x_end_pad = min(w_img, x + w + pad_x)
            y_end_pad = min(h_img, y + h + pad_y)

            crop = img_color[y_pad:y_end_pad, x_pad:x_end_pad]
            out_path = os.path.join(output_folder, f"zone_auto_k_{i+1:02d}.jpg")
            cv2.imwrite(out_path, crop, [cv2.IMWRITE_JPEG_QUALITY, 95])
            cropped_paths.append(out_path)

            if debug:
                cv2.rectangle(img_debug, (x_pad, y_pad), (x_end_pad, y_end_pad), (50, 50, 255), 5)
                cv2.putText(img_debug, f"Z{i+1}", (x_pad + 20, y_pad + 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (50, 50, 255), 6)

        if debug:
            debug_path = os.path.join(output_folder, "debug_auto_k_zones.jpg")
            cv2.imwrite(debug_path, img_debug)
            print(f"📊 Visualización de depuración guardada en: {debug_path}")

        print(f"\n✅ Proceso completado. Se guardaron {len(cropped_paths)} zonas.")
        return cropped_paths

# --- Cómo usar la clase ---
if __name__ == '__main__':
    cutter = AdvancedKMeansCutter()
    image_file = folleto
    output_dir = 'folleto_recortes_auto_kmeans'
    
    # Ya no necesitas especificar 'k'. El script lo encontrará por ti.
    paths = cutter.cut(image_file, output_dir, debug=True)


📸 Imagen cargada. Iniciando detección de zonas...
   -> Buscando k óptima entre 2 y 8...
   -> 'Codo' detectado. K óptima = 3
📊 Visualización de depuración guardada en: folleto_recortes_auto_kmeans\debug_auto_k_zones.jpg

✅ Proceso completado. Se guardaron 3 zonas.


### qwen3-vl:30b

In [3]:
qwen3 = OllamaOCR("qwen3-vl:30b")

In [4]:
tiles = qwen3.crop_by_zones(folleto, WINDOWED_CASA_LEY)

Successfully cropped 1 zones.


### Qwen 2.5

In [5]:
qwen = OllamaOCR("qwen2.5vl:32b")

In [3]:
tiles = qwen.crop_by_zones(folleto, WINDOWED_CASA_LEY)

Successfully cropped 1 zones.


In [6]:
tiles[0]

'C:\\Users\\angel.merino\\Documents\\GitHub\\jita\\datos\\casa_ley\\octubre_2025\\windowed\\pagina_01_01102025_r0_c0.png'

In [7]:
print(qwen.extract_text(tiles[0]))

El folleto que se muestra es una promoción de ofertas válidas solo para el martes 30 de septiembre de 2025. Se trata de una lista de productos de frutas, verduras y otros alimentos, con precios especiales. A continuación, se analiza el contenido del folleto:

### **Información General**
- **Fecha de la Promoción**: Martes 30 de septiembre de 2025.
- **Tipo de Productos**: Frutas, verduras, hortalizas y otros alimentos.
- **Restricciones**: Se menciona que hay un límite de compra por cliente para algunos productos (por ejemplo, "máximo 5 kilos" o "máximo 5 piezas").

### **Productos y Precios**
#### **Primera Fila**
1. **PAPA**
   - **Variedad**: Russet
   - **Presentación**: A granel
   - **Precio**: $19.90 por kilo
   - **Restricción**: Máximo 5 kilos por cliente

2. **CHILE**
   - **Variedad**: Anaheim
   - **Precio**: $49.90 por kilo
   - **Restricción**: Máximo 5 kilos por cliente

3. **CIRUELA**
   - **Variedad**: Roja España
   - **Precio**: $79.90 por kilo
   - **Restricción**: 

### Mistral small

In [14]:
mistral = OllamaOCR("mistral-small3.2:24b")

In [ ]:
print(mistral.extract_text(folleto))

'El folleto que has compartido es una promoción de un supermercado llamado "Ley", celebrando su aniversario. La promoción es válida solo el martes 30 de septiembre de 2014. Aquí hay un resumen detallado de las secciones y ofertas destacadas:\n\n### Sección de Frutas y Verduras\n- **Papa Russet**: $19.90 por 10 kg\n- **Cebolla**: $14.90 por 5 kg\n- **Pepino**: $24.90 por 6 piezas\n- **Jícama**: $19.90 por 3 kg\n- **Zanahoria**: $15.90 por 5 kg\n- **Naranja**: $19.90 por 5 kg\n- **Limón**: $29.90 por 5 kg\n- **Tomate**: $14.90 por 3 kg\n- **Cebolla Blanca**: $19.90 por 5 kg\n\n### Sección de Carnes\n- **Carne de Res para Asar**: $208.90 por kg\n- **Carne de Res para Asar (Oferta)**: $123.90 por kg\n- **Pollo Fresco**: $67.90 por 8 piezas\n- **Pulpa de Res**: $187.90 por kg\n- **Menudo**: $98.90 por kg\n- **Pierna y Muslo de Pollo**: $109.90 por kg\n- **Bistec de Res**: $99.90 por kg\n- **Camarón**: $196.90 por kg\n\n### Sección de Productos Lácteos y Yogur\n- **Yogur y Postres**: 3 x 2 a

### TiktokAPI

---

https://www.google.com/search?q=tiktok+api+for+developers&rlz=1C1ONGR_enMX1148MX1148&oq=tiktok+api+for+de&gs_lcrp=EgZjaHJvbWUqCQgBEAAYExiABDIGCAAQRRg5MgkIARAAGBMYgAQyCggCEAAYExgWGB4yDAgDEAAYChgTGBYYHjIKCAQQABiABBiiBDIHCAUQABjvBTIHCAYQABjvBTIGCAcQRRg80gEINzQ3N2owajSoAgCwAgE&sourceid=chrome&ie=UTF-8

In [ ]:
import requests
from datetime import datetime, timedelta
import json

class TikTokSearcher:
    def __init__(self, client_key, client_secret):
        self.client_key = client_key
        self.client_secret = client_secret
        self.access_token = None
        self.base_url = "https://open.tiktokapis.com/v2"
    
    def get_access_token(self):
        """Obtiene el token de acceso de la API de TikTok"""
        url = "https://open.tiktokapis.com/v2/oauth/token/"
        
        headers = {
            "Content-Type": "application/x-www-form-urlencoded",
            "Cache-Control": "no-cache"
        }
        
        data = {
            "client_key": self.client_key,
            "client_secret": self.client_secret,
            "grant_type": "client_credentials"
        }
        
        response = requests.post(url, headers=headers, data=data)
        
        if response.status_code == 200:
            self.access_token = response.json()["access_token"]
            print("✓ Token obtenido exitosamente")
            return True
        else:
            print(f"✗ Error obteniendo token: {response.status_code}")
            return False
    
    def search_videos(self, keyword, region="MX", max_results=20):
        """
        Busca videos en TikTok con filtros
        
        Args:
            keyword: Término de búsqueda
            region: Código de país (MX para México)
            max_results: Número máximo de resultados
        """
        if not self.access_token:
            print("Error: Primero obtén el access token")
            return None
        
        url = f"{self.base_url}/research/video/query/"
        
        # Fecha de hace 3 meses
        three_months_ago = datetime.now() - timedelta(days=90)
        start_date = three_months_ago.strftime("%Y%m%d")
        end_date = datetime.now().strftime("%Y%m%d")
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Content-Type": "application/json"
        }
        
        # Query con filtros de fecha y región
        query = {
            "query": {
                "and": [
                    {"field_name": "keyword", "field_values": [keyword]},
                    {"field_name": "region_code", "field_values": [region]}
                ]
            },
            "start_date": start_date,
            "end_date": end_date,
            "max_count": max_results
        }
        
        response = requests.post(url, headers=headers, json=query)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error en búsqueda: {response.status_code}")
            print(response.text)
            return None
    
    def extract_video_urls(self, search_results):
        """Extrae las URLs de los videos del resultado de búsqueda"""
        if not search_results or "data" not in search_results:
            return []
        
        videos = search_results["data"].get("videos", [])
        urls = []
        
        for video in videos:
            video_info = {
                "url": f"https://www.tiktok.com/@{video.get('username')}/video/{video.get('id')}",
                "titulo": video.get("video_description", "Sin título"),
                "likes": video.get("like_count", 0),
                "vistas": video.get("view_count", 0),
                "fecha": video.get("create_time", ""),
                "usuario": video.get("username", "")
            }
            urls.append(video_info)
        
        # Ordenar por relevancia (likes + vistas)
        urls.sort(key=lambda x: x["likes"] + x["vistas"], reverse=True)
        
        return urls


# ===== EJEMPLO DE USO =====
def main():
    # IMPORTANTE: Necesitas obtener tus credenciales en:
    # https://developers.tiktok.com/
    CLIENT_KEY = "TU_CLIENT_KEY_AQUI"
    CLIENT_SECRET = "TU_CLIENT_SECRET_AQUI"
    
    # Crear instancia del buscador
    searcher = TikTokSearcher(CLIENT_KEY, CLIENT_SECRET)
    
    # Obtener token de acceso
    if not searcher.get_access_token():
        print("No se pudo obtener el token. Verifica tus credenciales.")
        return
    
    # Buscar videos
    print("\nBuscando videos de 'carne asada en hermosillo'...")
    results = searcher.search_videos(
        keyword="carne asada hermosillo",
        region="MX",
        max_results=30
    )
    
    if results:
        # Extraer URLs
        videos = searcher.extract_video_urls(results)
        
        print(f"\n✓ Se encontraron {len(videos)} videos\n")
        print("=" * 80)
        
        for i, video in enumerate(videos, 1):
            print(f"\n{i}. {video['titulo'][:60]}...")
            print(f"   URL: {video['url']}")
            print(f"   Usuario: @{video['usuario']}")
            print(f"   👍 {video['likes']:,} likes | 👁 {video['vistas']:,} vistas")
            print(f"   📅 {video['fecha']}")
        
        # Guardar en archivo
        with open("tiktoks_carne_asada_hermosillo.json", "w", encoding="utf-8") as f:
            json.dump(videos, f, indent=2, ensure_ascii=False)
        
        print(f"\n✓ URLs guardadas en 'tiktoks_carne_asada_hermosillo.json'")


if __name__ == "__main__":
    main()

In [2]:
import instaloader
from datetime import datetime, timedelta
import json
import os

class InstagramSearcher:
    def __init__(self, username=None, password=None):
        """
        Inicializa el scraper de Instagram
        
        Args:
            username: Tu usuario de Instagram (opcional pero recomendado)
            password: Tu contraseña (opcional pero recomendado)
        """
        self.loader = instaloader.Instaloader(
            download_videos=True,
            download_video_thumbnails=False,
            download_geotags=False,
            download_comments=False,
            save_metadata=True,
            compress_json=False,
            post_metadata_txt_pattern=''
        )
        
        self.username = username
        self.logged_in = False
        
        # Intentar login si se proporcionaron credenciales
        if username and password:
            self.login(username, password)
    
    def login(self, username, password):
        """Inicia sesión en Instagram (recomendado para evitar rate limits)"""
        try:
            self.loader.login(username, password)
            self.logged_in = True
            print(f"✓ Sesión iniciada como @{username}")
            return True
        except Exception as e:
            print(f"✗ Error al iniciar sesión: {e}")
            print("Continuando sin login (limitaciones aplicarán)")
            return False
    
    def search_hashtag(self, hashtag, max_posts=50, days_ago=90):
        """
        Busca posts por hashtag
        
        Args:
            hashtag: Hashtag sin el símbolo # (ej: "carneasadahermosillo")
            max_posts: Número máximo de posts a obtener
            days_ago: Solo posts de los últimos X días
        """
        print(f"\nBuscando posts con #{hashtag}...")
        
        posts_data = []
        cutoff_date = datetime.now() - timedelta(days=days_ago)
        
        try:
            # Obtener posts del hashtag
            hashtag_obj = instaloader.Hashtag.from_name(self.loader.context, hashtag)
            posts = hashtag_obj.get_posts()
            
            count = 0
            for post in posts:
                # Verificar fecha
                if post.date_local < cutoff_date:
                    print(f"Alcanzado límite de {days_ago} días")
                    break
                
                # Recopilar información
                post_info = {
                    "url": f"https://www.instagram.com/p/{post.shortcode}/",
                    "shortcode": post.shortcode,
                    "fecha": post.date_local.strftime("%Y-%m-%d %H:%M:%S"),
                    "usuario": post.owner_username,
                    "likes": post.likes,
                    "comentarios": post.comments,
                    "descripcion": post.caption[:200] if post.caption else "",
                    "es_video": post.is_video,
                    "ubicacion": post.location.name if post.location else None
                }
                
                posts_data.append(post_info)
                count += 1
                
                print(f"  {count}. @{post.owner_username} - {post.likes} likes")
                
                if count >= max_posts:
                    break
            
            # Ordenar por relevancia (likes + comentarios)
            posts_data.sort(
                key=lambda x: x["likes"] + (x["comentarios"] * 2), 
                reverse=True
            )
            
            print(f"\n✓ Se encontraron {len(posts_data)} posts")
            return posts_data
            
        except Exception as e:
            print(f"✗ Error en búsqueda: {e}")
            return []
    
    def search_location(self, location_id, max_posts=50):
        """
        Busca posts por ubicación (necesitas el ID de la ubicación)
        
        Args:
            location_id: ID numérico de la ubicación en Instagram
            max_posts: Número máximo de posts
        """
        print(f"\nBuscando posts en ubicación {location_id}...")
        
        posts_data = []
        
        try:
            location = instaloader.ProfileLocation(self.loader.context, location_id)
            posts = location.get_posts()
            
            count = 0
            for post in posts:
                post_info = {
                    "url": f"https://www.instagram.com/p/{post.shortcode}/",
                    "shortcode": post.shortcode,
                    "usuario": post.owner_username,
                    "likes": post.likes
                }
                
                posts_data.append(post_info)
                count += 1
                
                if count >= max_posts:
                    break
            
            return posts_data
            
        except Exception as e:
            print(f"✗ Error: {e}")
            return []
    
    def download_post(self, url_or_shortcode, target_folder="descargas_instagram"):
        """
        Descarga un post específico
        
        Args:
            url_or_shortcode: URL completa o shortcode del post
            target_folder: Carpeta donde guardar
        """
        # Extraer shortcode de la URL si es necesario
        if "instagram.com" in url_or_shortcode:
            shortcode = url_or_shortcode.split("/p/")[1].split("/")[0]
        else:
            shortcode = url_or_shortcode
        
        try:
            # Crear carpeta si no existe
            os.makedirs(target_folder, exist_ok=True)
            
            # Obtener el post
            post = instaloader.Post.from_shortcode(self.loader.context, shortcode)
            
            # Descargar
            print(f"\nDescargando post de @{post.owner_username}...")
            self.loader.download_post(post, target=target_folder)
            
            print(f"✓ Post descargado en '{target_folder}'")
            return True
            
        except Exception as e:
            print(f"✗ Error descargando: {e}")
            return False
    
    def download_posts_from_list(self, posts_data, target_folder="descargas_instagram"):
        """Descarga múltiples posts de una lista"""
        print(f"\nDescargando {len(posts_data)} posts...")
        
        successful = 0
        for i, post in enumerate(posts_data, 1):
            print(f"\n[{i}/{len(posts_data)}]")
            if self.download_post(post["shortcode"], target_folder):
                successful += 1
        
        print(f"\n✓ {successful}/{len(posts_data)} posts descargados exitosamente")


# ===== EJEMPLOS DE USO =====

def ejemplo_busqueda_basica():
    """Búsqueda CON login (ahora obligatorio)"""
    print("=" * 80)
    print("EJEMPLO 1: Búsqueda con login")
    print("=" * 80)
    
    # IMPORTANTE: Necesitas proporcionar tus credenciales
    print("Instagram ahora requiere login para todas las búsquedas")
    print("Ingresa tus credenciales (o modifica el código directamente):\n")
    
    username = input("Usuario de Instagram: ").strip()
    password = input("Contraseña: ").strip()
    
    if not username or not password:
        print("✗ Se requieren credenciales. Saliendo...")
        return
    
    scraper = InstagramSearcher(username, password)
    
    if not scraper.logged_in:
        print("✗ No se pudo iniciar sesión. Verifica tus credenciales.")
        return
    
    # Buscar por hashtag
    posts = scraper.search_hashtag(
        hashtag="carneasadahermosillo",
        max_posts=20,
        days_ago=90
    )
    
    if posts:
        # Guardar resultados
        with open("instagram_carne_asada.json", "w", encoding="utf-8") as f:
            json.dump(posts, f, indent=2, ensure_ascii=False)
        
        print("\nTop 5 posts más populares:")
        print("-" * 80)
        for i, post in enumerate(posts[:5], 1):
            print(f"\n{i}. @{post['usuario']}")
            print(f"   {post['url']}")
            print(f"   ❤️ {post['likes']:,} likes | 💬 {post['comentarios']} comentarios")
            print(f"   📅 {post['fecha']}")


def ejemplo_con_login():
    """Búsqueda con login (más confiable y sin rate limits)"""
    print("=" * 80)
    print("EJEMPLO 2: Búsqueda con login")
    print("=" * 80)
    
    # IMPORTANTE: Usa tus credenciales reales
    USERNAME = "tu_usuario"
    PASSWORD = "tu_password"
    
    scraper = InstagramSearcher(USERNAME, PASSWORD)
    
    # Buscar posts
    posts = scraper.search_hashtag(
        hashtag="hermosillo",
        max_posts=30,
        days_ago=30
    )
    
    if posts:
        print(f"\n¿Descargar los {len(posts)} posts? (s/n)")
        # En producción, descomenta para confirmar:
        # if input().lower() == 's':
        #     scraper.download_posts_from_list(posts[:10])  # Descargar solo top 10


def ejemplo_descarga_directa():
    """Descargar un post específico por URL"""
    print("=" * 80)
    print("EJEMPLO 3: Descarga directa de post")
    print("=" * 80)
    
    scraper = InstagramSearcher()
    
    # Descargar un post específico
    url = "https://www.instagram.com/p/EJEMPLO123/"
    scraper.download_post(url)


# ===== EJECUTAR =====
if __name__ == "__main__":
    # Descomentar el ejemplo que quieras probar:
    
    ejemplo_busqueda_basica()
    # ejemplo_con_login()
    # ejemplo_descarga_directa()
    
    print("\n" + "=" * 80)
    print("✓ Proceso completado")
    print("=" * 80)

EJEMPLO 1: Búsqueda con login
Instagram ahora requiere login para todas las búsquedas
Ingresa tus credenciales (o modifica el código directamente):

✗ Se requieren credenciales. Saliendo...

✓ Proceso completado


In [ ]:
import requests
import json
from datetime import datetime, timedelta
import urllib.parse

class InstagramGraphAPI:
    """
    Cliente para la Instagram Graph API (API oficial de Meta)
    
    IMPORTANTE: Esta API tiene limitaciones:
    - Solo puedes acceder a cuentas de Instagram Business/Creator
    - Necesitas que el usuario te de permiso explícito
    - NO permite búsquedas generales por hashtag como usuario regular
    - Útil para gestionar tu propia cuenta o cuentas autorizadas
    """
    
    def __init__(self, access_token):
        """
        Args:
            access_token: Token de acceso de Instagram Graph API
        """
        self.access_token = access_token
        self.base_url = "https://graph.facebook.com/v19.0"
    
    def get_user_id(self):
        """Obtiene el ID del usuario autenticado"""
        url = f"{self.base_url}/me"
        params = {
            "fields": "id,username",
            "access_token": self.access_token
        }
        
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            print(f"✓ Usuario autenticado: @{data.get('username')} (ID: {data.get('id')})")
            return data.get('id')
        else:
            print(f"✗ Error: {response.status_code}")
            print(response.text)
            return None
    
    def get_user_media(self, user_id=None, limit=25):
        """
        Obtiene los posts del usuario autenticado o de un usuario específico
        
        Args:
            user_id: ID del usuario (None para el usuario autenticado)
            limit: Número máximo de posts a obtener
        """
        if not user_id:
            user_id = "me"
        
        url = f"{self.base_url}/{user_id}/media"
        params = {
            "fields": "id,caption,media_type,media_url,permalink,thumbnail_url,timestamp,like_count,comments_count,username",
            "access_token": self.access_token,
            "limit": limit
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            posts = data.get('data', [])
            print(f"✓ Se obtuvieron {len(posts)} posts")
            return posts
        else:
            print(f"✗ Error: {response.status_code}")
            print(response.text)
            return []
    
    def search_hashtag(self, hashtag, user_id=None):
        """
        Busca información de un hashtag
        
        NOTA: Esta función solo devuelve metadata del hashtag, NO los posts
        Para obtener posts de hashtags necesitas Instagram Basic Display API
        y permisos especiales
        
        Args:
            hashtag: Hashtag sin el símbolo #
            user_id: ID del usuario de Instagram Business
        """
        if not user_id:
            user_id = self.get_user_id()
        
        # Primero buscar el ID del hashtag
        url = f"{self.base_url}/ig_hashtag_search"
        params = {
            "user_id": user_id,
            "q": hashtag,
            "access_token": self.access_token
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if data.get('data'):
                hashtag_id = data['data'][0]['id']
                print(f"✓ Hashtag encontrado: #{hashtag} (ID: {hashtag_id})")
                
                # Obtener información del hashtag
                return self.get_hashtag_info(hashtag_id)
            else:
                print(f"✗ Hashtag #{hashtag} no encontrado")
                return None
        else:
            print(f"✗ Error: {response.status_code}")
            print(response.text)
            return None
    
    def get_hashtag_info(self, hashtag_id):
        """Obtiene información de un hashtag específico"""
        url = f"{self.base_url}/{hashtag_id}"
        params = {
            "fields": "id,name",
            "access_token": self.access_token
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"✗ Error: {response.status_code}")
            return None
    
    def get_recent_hashtag_media(self, hashtag_id, user_id, limit=25):
        """
        Obtiene posts recientes de un hashtag
        
        REQUIERE: Cuenta de Instagram Business y permisos específicos
        """
        url = f"{self.base_url}/{hashtag_id}/recent_media"
        params = {
            "user_id": user_id,
            "fields": "id,caption,media_type,media_url,permalink,timestamp,like_count,comments_count",
            "access_token": self.access_token,
            "limit": limit
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            return data.get('data', [])
        else:
            print(f"✗ Error: {response.status_code}")
            print(response.text)
            return []
    
    def download_media(self, media_url, filename):
        """Descarga una imagen o video desde su URL"""
        try:
            response = requests.get(media_url, stream=True)
            if response.status_code == 200:
                with open(filename, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"✓ Descargado: {filename}")
                return True
            else:
                print(f"✗ Error descargando: {response.status_code}")
                return False
        except Exception as e:
            print(f"✗ Error: {e}")
            return False


# ===== CÓMO OBTENER UN ACCESS TOKEN =====

def obtener_access_token_instrucciones():
    """
    Instrucciones para obtener un Access Token de Instagram Graph API
    """
    print("""
    ╔════════════════════════════════════════════════════════════════════════════╗
    ║          CÓMO OBTENER UN ACCESS TOKEN DE INSTAGRAM GRAPH API              ║
    ╚════════════════════════════════════════════════════════════════════════════╝
    
    PASO 1: Crear una App de Facebook
    ────────────────────────────────────────────────────────────────────────────
    1. Ve a https://developers.facebook.com/
    2. Crea una cuenta de desarrollador si no tienes
    3. Crea una nueva aplicación
    4. Selecciona "Consumer" como tipo de app
    5. Dale un nombre a tu app
    
    PASO 2: Configurar Instagram Graph API
    ────────────────────────────────────────────────────────────────────────────
    1. En tu app, ve a "Add Product"
    2. Agrega "Instagram Graph API"
    3. Ve a Settings > Basic y copia tu "App ID" y "App Secret"
    
    PASO 3: Convertir tu cuenta de Instagram a Business
    ────────────────────────────────────────────────────────────────────────────
    1. En la app de Instagram móvil:
       - Ve a tu perfil
       - Settings > Account > Switch to Professional Account
       - Elige "Business"
    2. Conecta tu cuenta de Instagram a una página de Facebook
    
    PASO 4: Obtener un Access Token
    ────────────────────────────────────────────────────────────────────────────
    OPCIÓN A - Token de corta duración (más fácil, expira en 1 hora):
    
    1. Ve a Graph API Explorer: https://developers.facebook.com/tools/explorer/
    2. Selecciona tu aplicación
    3. Click en "Generate Access Token"
    4. Selecciona los permisos:
       - instagram_basic
       - instagram_content_publish
       - pages_read_engagement
       - pages_show_list
    5. Copia el token generado
    
    OPCIÓN B - Token de larga duración (recomendado, dura 60 días):
    
    1. Obtén primero un token de corta duración (Opción A)
    2. Usa esta URL (reemplaza los valores):
    
    https://graph.facebook.com/v19.0/oauth/access_token?
      grant_type=fb_exchange_token&
      client_id={TU_APP_ID}&
      client_secret={TU_APP_SECRET}&
      fb_exchange_token={TU_TOKEN_CORTO}
    
    3. El response tendrá tu token de larga duración
    
    PASO 5: Probar tu token
    ────────────────────────────────────────────────────────────────────────────
    Prueba con esta URL en tu navegador:
    
    https://graph.facebook.com/v19.0/me?fields=id,username&access_token={TU_TOKEN}
    
    ╔════════════════════════════════════════════════════════════════════════════╗
    ║                              LIMITACIONES                                  ║
    ╚════════════════════════════════════════════════════════════════════════════╝
    
    ⚠️  IMPORTANTE:
    - Solo funciona con cuentas de Instagram BUSINESS o CREATOR
    - NO puedes buscar posts de otras personas sin su permiso
    - NO puedes hacer búsquedas generales por hashtag como usuario normal
    - Solo puedes gestionar TU propia cuenta o cuentas que te den acceso
    
    Para búsquedas generales de contenido público, necesitas usar web scraping
    (como el código anterior con instaloader + login).
    """)


# ===== EJEMPLOS DE USO =====

def ejemplo_api_oficial():
    """Ejemplo usando la API oficial de Meta"""
    print("=" * 80)
    print("EJEMPLO: Instagram Graph API (Oficial)")
    print("=" * 80)
    
    # IMPORTANTE: Reemplaza con tu token real
    ACCESS_TOKEN = "TU_ACCESS_TOKEN_AQUI"
    
    if ACCESS_TOKEN == "TU_ACCESS_TOKEN_AQUI":
        print("\n⚠️  Necesitas configurar tu ACCESS_TOKEN primero\n")
        obtener_access_token_instrucciones()
        return
    
    api = InstagramGraphAPI(ACCESS_TOKEN)
    
    # Obtener tus propios posts
    print("\n1. Obteniendo tus posts...")
    posts = api.get_user_media(limit=10)
    
    if posts:
        print(f"\nSe encontraron {len(posts)} posts:\n")
        print("-" * 80)
        
        for i, post in enumerate(posts, 1):
            print(f"\n{i}. {post.get('media_type', 'Unknown')}")
            print(f"   URL: {post.get('permalink', 'N/A')}")
            print(f"   Caption: {post.get('caption', 'Sin caption')[:60]}...")
            print(f"   ❤️ {post.get('like_count', 0)} likes")
            print(f"   📅 {post.get('timestamp', 'N/A')}")
        
        # Guardar resultados
        with open("instagram_mis_posts.json", "w", encoding="utf-8") as f:
            json.dump(posts, f, indent=2, ensure_ascii=False)
        
        print(f"\n✓ Datos guardados en 'instagram_mis_posts.json'")
        
        # Descargar primera imagen/video
        if posts[0].get('media_url'):
            print(f"\n2. Descargando primer post...")
            filename = f"instagram_{posts[0]['id']}.jpg"
            api.download_media(posts[0]['media_url'], filename)


def ejemplo_comparacion():
    """Muestra la diferencia entre ambos métodos"""
    print("""
    ╔════════════════════════════════════════════════════════════════════════════╗
    ║                    COMPARACIÓN: API vs Web Scraping                        ║
    ╚════════════════════════════════════════════════════════════════════════════╝
    
    ┌─────────────────────────────────────────────────────────────────────────┐
    │ INSTAGRAM GRAPH API (Oficial de Meta)                                   │
    ├─────────────────────────────────────────────────────────────────────────┤
    │ ✅ Ventajas:                                                             │
    │    • Legal y oficial                                                     │
    │    • Estable, no cambia constantemente                                   │
    │    • No hay riesgo de bloqueo de cuenta                                  │
    │    • Datos estructurados y confiables                                    │
    │                                                                           │
    │ ❌ Desventajas:                                                          │
    │    • Solo para cuentas Business/Creator                                  │
    │    • NO permite búsquedas públicas generales                             │
    │    • Solo accedes a TU contenido o cuentas autorizadas                   │
    │    • Configuración compleja (tokens, apps, permisos)                     │
    │    • NO puedes buscar "carne asada hermosillo" de otros usuarios         │
    └─────────────────────────────────────────────────────────────────────────┘
    
    ┌─────────────────────────────────────────────────────────────────────────┐
    │ WEB SCRAPING (Instaloader + Login)                                      │
    ├─────────────────────────────────────────────────────────────────────────┤
    │ ✅ Ventajas:                                                             │
    │    • Puedes buscar contenido público de cualquier persona                │
    │    • Búsqueda por hashtags (#carneasadahermosillo)                       │
    │    • Búsqueda por ubicaciones                                            │
    │    • Más flexible para exploración                                       │
    │                                                                           │
    │ ❌ Desventajas:                                                          │
    │    • Requiere login obligatorio                                          │
    │    • Riesgo de bloqueo temporal si haces muchas requests                 │
    │    • Instagram puede cambiar su estructura y romper el código            │
    │    • Técnicamente va contra los términos de servicio                     │
    │    • Necesitas usar tu cuenta personal (o crear una secundaria)          │
    └─────────────────────────────────────────────────────────────────────────┘
    
    RECOMENDACIÓN PARA TU CASO:
    ═══════════════════════════════════════════════════════════════════════════
    
    Para buscar "carne asada en hermosillo" de otros usuarios:
    → USA WEB SCRAPING (instaloader con login)
    
    Para gestionar tu propia cuenta de negocio:
    → USA LA API OFICIAL (Instagram Graph API)
    """)


if __name__ == "__main__":
    # Mostrar comparación primero
    ejemplo_comparacion()
    
    print("\n" + "=" * 80)
    input("Presiona Enter para ver las instrucciones de la API oficial...")
    print("=" * 80 + "\n")
    
    # Mostrar instrucciones
    obtener_access_token_instrucciones()
    
    # Si quieres probar el código:
    # ejemplo_api_oficial()